### Dekorátorok

In [ ]:
# Ezt egy régi kaggle notebookomból vettem kölcsön, ezért Angol minden is...
# https://www.kaggle.com/code/lszlbebesi/the-python-interview-v-1-0#Decorators


# Let's define the decorator
def random_decorator(func):
    """
    The input is the func (a function)
    I add args and kwarg because I anticipate that the function may have input arguments
    I encourage the reader to try this construct without the addition of args and kwargs on the decorator_tester_function2 which has an input
    I promise its not gonna work without args and kwargs
    """

    def inner_function(*args, **kwargs):
        print("Random decorator (behaviour before function execution)")
        # calling the actual function now
        # inside the wrapper function.
        func(*args, **kwargs)
        print("Behavior after execution")

    return inner_function


def decorator_tester_function():
    print("Decorator tester function")


# Lets call the function 1st the simple was
print("Simple function call: ")
decorator_tester_function()
print("\n")

# Now I redefine the same function
# To call it again


@random_decorator
def decorator_tester_function():
    print("Decorator tester function")


print("Decorator function call: ")
decorator_tester_function()
print("\n")


@random_decorator
def decorator_tester_function2(message):
    """decorator_tester_function2 original"""
    print(message)


decorator_tester_function2("Function + Decorator + inputs")

Simple function call: 
Decorator tester function


Decorator function call: 
Random decorator (behaviour before function execution)
Decorator tester function
Behavior after execution


Random decorator (behaviour before function execution)
Function + Decorator + inputs
Behavior after execution


#### Mi a probléma a random_decoratorral?

In [10]:
import functools


def random_decorator_fixed(func):
    """
    The input is the func (a function).
    I add args and kwargs because I anticipate that the function may have input arguments.
    I encourage the reader to try this construct without the addition of args and kwargs on
    the decorator_tester_function2 which has an input.
    I promise it's not gonna work without args and kwargs.
    """

    @functools.wraps(func)  # This ensures the function metadata is preserved
    def inner_function(*args, **kwargs):
        print("Random decorator (behavior before function execution)")
        result = func(*args, **kwargs)  # Call the actual function
        print("Behavior after execution")
        return result  # Ensure the return value is passed through

    return inner_function


@random_decorator_fixed
def decorator_tester_function_fixed(message):
    """decorator_tester_function_fixed explanation"""
    print(message)
    

print(f"Original function name (fixed): {decorator_tester_function_fixed.__name__}")
print(f"Original function docstring (fixed): {decorator_tester_function_fixed.__doc__}")


print(f"Original function name (Original): {decorator_tester_function2.__name__}")
print(f"Original function docstring (Original): {decorator_tester_function2.__doc__}")


decorator_tester_function2("!!")

Original function name (fixed): decorator_tester_function_fixed
Original function docstring (fixed): decorator_tester_function_fixed explanation
Original function name (Original): inner_function
Original function docstring (Original): None
Random decorator (behaviour before function execution)
!!
Behavior after execution


#### A functools wrapper segítségével már tartható az eredeti függvény metaadata (docstring + név)

Mire jók a dekorátorok?
<br>
<li><b>A dekorátorok új funkciókat, új fajta működést biztosítanak meglevő függvényekhez, metódusokhoz anélkül, hogy át kellene azokat írni...</b></li>
<li>Dekorátorokkal megvalósíthatunk logolást, ami segíthet a debug-olásban.</li>
<li>A dekorátorok alkalmasak lehetnek jogosultságok kezelésére (lásd következő kurzus, Django alapok...)</li>
<li>Cache-elésre is alkalmasak lehetnek a dekorátorok (lásd lentebb a feladatot)</li>
<li>Az argumentum és bemenetek ellenőrzésére is írható dekorátor</li>



### Generátorok (és python iterátorok)

In [ ]:
class ExampleIterator:
    def __init__(self, start, end):
        self.current = start  # legyen egy kezdő érték
        self.end = end        # és legyen egy befejező érték
    
    def __iter__(self):
        return self

    def __next__(self):
        if self.current <= self.end:
            value = self.current
            self.current += 1
            return value
        else:
            raise StopIteration

numbers = ExampleIterator(1, 5)

for number in numbers:
    print(number)

# Egy Python iterátor egy olyan objektum, ami lehetővé teszi, hogy a for ciklus segítségével iteráljunk rajta,
# vagy esetleg közvetlenül a next() függvénnyel hívjunk meg egy új elemet.

# A Pythonban egy objektum akkor iterátor, ha rendelkezik két alapvető metódussal:

# __iter__(): Ez a metódus visszaadja az iterátort. 
# Az iterátorok számára a __iter__ metódus gyakran az objektumot magát adja vissza, így lehetővé teszi a következő értékek lekérését.
# __next__(): Ez a metódus a következő elemet adja vissza. 
# Ha az iterátor elérte a végső elemet, akkor a StopIteration kivételt dobja, jelezve, hogy nincs több elem.
# Másképpen kell egy megállási feltétel (e.g. fentebb a self.end value-t definiáltuk)

1
2
3
4
5


### Elég ritka, hogy saját iterátort írogat a programozó, helyette inkább generátorokat használunk

A python generátor egy olyan iterátor, ami lusta kiértékelést alkalmaz (lazy eval), értéket csak akkor számít ki, amikor arra szükség van.
Alapvetően kétféle módon hozható létre:
1. generátor kifejezéssel (generator experssion)
2. generátor függvénnyel (generator function)

A lazy evaluation sokat segít, egyrészt memória hatékony lesz így a generátor (mivel nem értékeljük ki az összes elemet), másrészt a teljesítményre is ez pozitívan hat... (nem telítődik a memória mindennel is...)

In [15]:
#generátor kifejezéssel
exp_generator = (2**x for x in range(5))
print(f"Ez valóban generátor {type(exp_generator)}")

# nézük meg mi van akkor ha szögletes zárójelbe írjuk...
exp_generator_list = [2**x for x in range(5)]
print(f"Ez valóban generátor {type(exp_generator_list)}")
# listává alakul... Viszont a () zárójelek között nem lesz belőle tuple,
# hanem marad generátor :)

Ez valóban generátor <class 'generator'>
Ez valóban generátor <class 'list'>


In [ ]:
# generátor függvényes megoldás
def exp_numbers(n):
    for i in range(n):
        yield 2 ** i

exp_gen = exp_numbers(5)

for num in exp_gen:
    print(num)

# mit tud a yield function? Miért nem return?

1
2
4
8
16


Amikor a generátor eléri a yield kulcsszót, a függvény állapota "suspended" és nem "stopped" mint a return esetében. Így a következő híváskor a generátor folytatja ott, ahol az előző yield után abbahagyta, így nem kell az összes értéket egyszerre kiszámítani és tárolni.